# data_world.csv

Energia grupowana po regionie i czasie + informacje o regionie + liczba trzęsień. LSTM, Dense, XGB

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
import datetime as dt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, f1_score
import tensorflow as tf
import xgboost as xgb
warnings.filterwarnings('ignore')

2023-10-03 15:17:52.834002: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-03 15:17:54.022241: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
df = pd.read_csv('../../data/data_world.csv')
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,1930-12-08T08:01:02.000Z,23.261,120.277,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1930-12-03T18:51:47.000Z,18.233,96.298,10.0,7.4,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1930-12-02T07:01:30.000Z,25.854,98.356,35.0,6.2,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1930-11-28T07:32:56.000Z,18.779,-106.767,15.0,6.3,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1930-11-25T19:02:53.000Z,35.050,139.129,15.0,6.9,mw,NaN,NaN,NaN,NaN,...,2015-05-13T18:52:43.000Z,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df = df[["time", "latitude", "longitude", "depth", "mag"]]

In [4]:
df["time"] = pd.to_datetime(df["time"], format="%Y-%m-%dT%H:%M:%S.%fZ")
df

,time,latitude,longitude,depth,mag
0,1930-12-08 08:01:02.000,23.2610,120.2770,15.00,6.3
1,1930-12-03 18:51:47.000,18.2330,96.2980,10.00,7.4
2,1930-12-02 07:01:30.000,25.8540,98.3560,35.00,6.2
3,1930-11-28 07:32:56.000,18.7790,-106.7670,15.00,6.3
4,1930-11-25 19:02:53.000,35.0500,139.1290,15.00,6.9
...,...,...,...,...,...
797041,2018-09-01 01:14:38.230,-30.4830,-177.9279,43.90,4.3
797042,2018-09-01 01:07:59.120,-10.7558,124.3621,10.00,4.0
797043,2018-09-01 01:00:13.810,-5.5167,147.1735,217.56,4.6
797044,2018-09-01 00:27:11.440,46.8819,155.6566,10.00,4.3


In [5]:
time_cut = dt.datetime(1973, 1, 1)
df = df[df["time"] > time_cut]
df["energy"] = 10**(1.44*df["mag"]+5.24)
df["label"] = np.where(df["mag"] > 5, 1, 0)

In [6]:
time_split = 30

In [7]:
# group the data into 30 days intervals
df["time"] = df["time"].dt.floor('d')
df["time"] = df["time"] - np.min(df["time"])
df["time"] = df["time"].dt.days // time_split
df["time"] = df["time"].astype(int)

In [8]:
geo_split = 10

In [9]:
df["latitude"] = df["latitude"] // geo_split
df["latitude"] = df["latitude"] + np.abs(np.min(df["latitude"]))

df["longitude"] = df["longitude"] // geo_split
df["longitude"] = df["longitude"] + np.abs(np.min(df["longitude"]))

In [10]:
df.loc[df["depth"] < 0, "depth"] = 0

In [11]:
df["depth"] = df["depth"] // 100

In [12]:
# add count of grouped data in each group
df_agg = df.groupby(["latitude", "longitude", "depth", "time"]).agg({"energy": "sum", "label": "sum"}).reset_index()
df_agg["count"] = df.groupby(["latitude", "longitude", "depth", "time"]).count().reset_index()["mag"]
df_agg

,latitude,longitude,depth,time,energy,label,count
0,0.0,13.0,0.0,268,1.018591e+12,0,1
1,0.0,22.0,0.0,482,3.767038e+11,0,1
2,0.0,31.0,0.0,375,5.248075e+11,0,1
3,0.0,31.0,0.0,473,7.311391e+11,0,1
4,0.0,33.0,0.0,280,2.703958e+11,0,1
...,...,...,...,...,...,...,...
106225,17.0,30.0,0.0,487,1.018591e+12,0,1
106226,17.0,30.0,0.0,489,3.767038e+11,0,1
106227,17.0,30.0,0.0,509,7.311391e+11,0,1
106228,17.0,30.0,0.0,538,3.767038e+11,0,1


In [13]:
df_agg["label"] = np.where(df_agg["label"] > 0, 1, 0)

In [14]:
def make_ds(df_agg, block_size):
    dfs_train, dfs_val = [], []
    for i in df_agg["latitude"].unique():
        for j in df_agg["longitude"].unique():
            for d in df_agg["depth"].unique():
                tmp = df_agg[(df_agg["latitude"] == i) & (df_agg["longitude"] == j) & (df_agg["depth"] == d)]
                if not tmp.empty:
                    start = max(tmp["time"].min() - block_size, 0)
                    end = min(tmp["time"].max() + block_size, df_agg["time"].max())
                    tmp = tmp.set_index("time").reindex(range(start, end)).fillna(0).rename_axis('time')
                    tmp["label"] = tmp["label"].shift(-1)
                    tmp["latitude"] = i
                    tmp["longitude"] = j
                    tmp["depth"] = d
                    n = int(0.8 * len(tmp))
                    df_train = tmp[:n]
                    df_val = tmp[n:]    
                    dfs_train.append(df_train)
                    dfs_val.append(df_val)
    df_final_train = pd.concat(dfs_train)
    df_final_val = pd.concat(dfs_val)
    return df_final_train, df_final_val

In [15]:
block_size = 16
df_train, df_val = make_ds(df_agg, block_size)

In [16]:
dfs_train, dfs_val = [], []
for i in df_train["latitude"].unique():
    for j in df_train["longitude"].unique():
        for d in df_train["depth"].unique():
            tmp1 = df_train[(df_train["latitude"] == i) & (df_train["longitude"] == j) & (df_train["depth"] == d)]
            tmp2 = df_val[(df_val["latitude"] == i) & (df_val["longitude"] == j) & (df_val["depth"] == d)]
            if not tmp1.empty and not tmp2.empty:
                scaler = MinMaxScaler()
                tmp1["energy0"] = scaler.fit_transform(tmp1[["energy"]])
                tmp2["energy0"] = scaler.transform(tmp2[["energy"]])
                for idx in range(1, block_size):
                    tmp1["count" + str(idx)] = tmp1["count"].shift(idx)
                    tmp2["count" + str(idx)] = tmp2["count"].shift(idx)
                    tmp1["energy" + str(idx)] = tmp1["energy0"].shift(idx)
                    tmp2["energy" + str(idx)] = tmp2["energy0"].shift(idx)
                dfs_train.append(tmp1)
                dfs_val.append(tmp2)
df_final_train = pd.concat(dfs_train)
df_final_val = pd.concat(dfs_val)

In [17]:
df_final_train.head()

,latitude,longitude,depth,energy,label,count,energy0,count1,energy1,count2,...,count11,energy11,count12,energy12,count13,energy13,count14,energy14,count15,energy15
time,,,,,,,,,,,,,,,,,,,,,
252,0.0,13.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
253,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
254,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
255,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
256,0.0,13.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
df_final_train.dropna(inplace=True)
df_final_val.dropna(inplace=True)

In [93]:
x_train = df_final_train.drop(["label", "longitude", "latitude", "energy", "depth"], axis=1).to_numpy()
y_train = df_final_train["label"].to_numpy()
x_val = df_final_val.drop(["label", "longitude", "latitude", "energy", "depth"], axis=1).to_numpy()
y_val = df_final_val["label"].to_numpy()

In [94]:
x_train = x_train.reshape(-1, block_size, 2)
x_val = x_val.reshape(-1, block_size, 2)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

In [95]:
# add anoother column whith 1 - label
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [96]:
model = tf.keras.Sequential([
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(512, input_shape=(block_size, 2), return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(256)),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dense(2),
])

In [97]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [98]:
model.fit(x_train, y_train, epochs=20, batch_size=512, validation_data=(x_val, y_val))

Epoch 1/20
507/507 [==============================] - 61s 102ms/step - loss: 0.2357 - accuracy: 0.9070 - f1_score: 0.5344 - val_loss: 0.2586 - val_accuracy: 0.9011 - val_f1_score: 0.4818
Epoch 2/20
507/507 [==============================] - 58s 114ms/step - loss: 0.2291 - accuracy: 0.9091 - f1_score: 0.5322 - val_loss: 0.3085 - val_accuracy: 0.8924 - val_f1_score: 0.5604
Epoch 3/20
507/507 [==============================] - 62s 122ms/step - loss: 0.2288 - accuracy: 0.9100 - f1_score: 0.5491 - val_loss: 0.2409 - val_accuracy: 0.9043 - val_f1_score: 0.6272
Epoch 4/20
507/507 [==============================] - 63s 124ms/step - loss: 0.2292 - accuracy: 0.9104 - f1_score: 0.5548 - val_loss: 0.2461 - val_accuracy: 0.9071 - val_f1_score: 0.5338
Epoch 5/20
507/507 [==============================] - 70s 138ms/step - loss: 0.2298 - accuracy: 0.9108 - f1_score: 0.5565 - val_loss: 0.2397 - val_accuracy: 0.9082 - val_f1_score: 0.6331
Epoch 6/20
507/507 [==============================] - 79s 156ms/s

In [129]:
x_train = df_final_train.drop(["label", "longitude", "latitude", "energy", "depth"], axis=1).to_numpy()
y_train = df_final_train["label"].to_numpy()
x_val = df_final_val.drop(["label", "longitude", "latitude", "energy", "depth"], axis=1).to_numpy()
y_val = df_final_val["label"].to_numpy()

In [130]:
x_train = x_train.reshape(-1, block_size, 2)
x_val = x_val.reshape(-1, block_size, 2)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

In [131]:
# add anoother column whith 1 - label
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [132]:
x_pos_train = df_final_train[["longitude", "latitude", "depth"]].to_numpy()
x_pos_val = df_final_val[["longitude", "latitude", "depth"]].to_numpy()

In [133]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed, vocab_size):
        super(MyModel, self).__init__()
        self.n_embed = n_embed
        self.loc_emb_x = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.loc_emb_y = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.loc_emb_z = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.conc = tf.keras.layers.Concatenate()
        self.dense_h = tf.keras.layers.Dense(3 * n_embed)
        self.dense_c = tf.keras.layers.Dense(3 * n_embed)
        self.lstm1 = tf.keras.layers.LSTM(n_embed * 3, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed * 3)
        self.dense = tf.keras.layers.Dense(n_embed, activation='relu')
        self.out = tf.keras.layers.Dense(2)
        

    def call(self, inputs):
        loc, timestep = inputs
        x = self.loc_emb_x(loc[:, 0])
        y = self.loc_emb_y(loc[:, 1])
        z = self.loc_emb_z(loc[:, 2])
        x_loc = self.conc([x, y, z])
        x_loc = tf.reshape(x_loc, [-1, 3 * self.n_embed])
        h = self.dense_h(x_loc)
        c = self.dense_c(x_loc)
        x = self.lstm1(timestep, initial_state=[c, h])
        x = self.lstm2(x, initial_state=[c, h])
        x = self.dense(x)
        return self.out(x)

In [134]:
n_embed = 256
vocab_size = 2000
model = MyModel(n_embed, vocab_size)

In [135]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [136]:
model.fit([x_pos_train, x_train], y_train, epochs=20, batch_size=512, validation_data=([x_pos_val, x_val], y_val))

Epoch 1/20
507/507 [==============================] - 80s 136ms/step - loss: 0.2221 - accuracy: 0.9179 - f1_score: 0.6428 - val_loss: 0.2160 - val_accuracy: 0.9225 - val_f1_score: 0.6834
Epoch 2/20
507/507 [==============================] - 80s 158ms/step - loss: 0.2486 - accuracy: 0.9197 - f1_score: 0.6669 - val_loss: 0.2296 - val_accuracy: 0.9154 - val_f1_score: 0.6962
Epoch 3/20
507/507 [==============================] - 97s 191ms/step - loss: 0.2141 - accuracy: 0.9243 - f1_score: 0.6850 - val_loss: 0.2187 - val_accuracy: 0.9203 - val_f1_score: 0.7072
Epoch 4/20
507/507 [==============================] - 113s 224ms/step - loss: 0.2080 - accuracy: 0.9252 - f1_score: 0.6959 - val_loss: 0.2153 - val_accuracy: 0.9208 - val_f1_score: 0.7045
Epoch 5/20
507/507 [==============================] - 164s 324ms/step - loss: 0.2045 - accuracy: 0.9262 - f1_score: 0.7010 - val_loss: 0.2106 - val_accuracy: 0.9218 - val_f1_score: 0.6905
Epoch 6/20
507/507 [==============================] - 215s 425m

In [121]:
x_train = df_final_train.drop(["label", "longitude", "latitude", "energy", "depth"], axis=1).to_numpy()
y_train = df_final_train["label"].to_numpy()
x_val = df_final_val.drop(["label", "longitude", "latitude", "energy", "depth"], axis=1).to_numpy()
y_val = df_final_val["label"].to_numpy()

In [122]:
x_train = x_train.reshape(-1, block_size, 2)
x_val = x_val.reshape(-1, block_size, 2)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

In [123]:
# add anoother column whith 1 - label
y_train = np.concatenate((y_train, 1 - y_train), axis=1)
y_val = np.concatenate((y_val, 1 - y_val), axis=1)

In [124]:
x_pos_train = df_final_train[["longitude", "latitude", "depth"]].to_numpy()
x_pos_val = df_final_val[["longitude", "latitude", "depth"]].to_numpy()

In [125]:
class MyModel(tf.keras.Model):
    def __init__(self, n_embed, vocab_size):
        super(MyModel, self).__init__()
        self.n_embed = n_embed
        self.h_loc_emb_x = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.h_loc_emb_y = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.h_loc_emb_z = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.c_loc_emb_x = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.c_loc_emb_y = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.c_loc_emb_z = tf.keras.layers.Embedding(vocab_size, n_embed)
        self.conc_h = tf.keras.layers.Concatenate()
        self.conc_c = tf.keras.layers.Concatenate()
        self.lstm1 = tf.keras.layers.LSTM(n_embed * 3, return_sequences=True)
        self.lstm2 = tf.keras.layers.LSTM(n_embed * 3)
        self.dense = tf.keras.layers.Dense(n_embed, activation='relu')
        self.out = tf.keras.layers.Dense(2)
        

    def call(self, inputs):
        loc, timestep = inputs
        x = self.h_loc_emb_x(loc[:, 0])
        y = self.h_loc_emb_y(loc[:, 1])
        z = self.h_loc_emb_z(loc[:, 2])
        x_loc_h = self.conc_h([x, y, z])
        x_loc_h = tf.reshape(x_loc_h, [-1, 3 * self.n_embed])
        x = self.c_loc_emb_x(loc[:, 0])
        y = self.c_loc_emb_y(loc[:, 1])
        z = self.c_loc_emb_z(loc[:, 2])
        x_loc_c = self.conc_c([x, y, z])
        x_loc_c = tf.reshape(x_loc_c, [-1, 3 * self.n_embed])
        x = self.lstm1(timestep, initial_state=[x_loc_c, x_loc_h])
        x = self.lstm2(x, initial_state=[x_loc_c, x_loc_h])
        x = self.dense(x)
        return self.out(x)

In [126]:
n_embed = 256
vocab_size = 2000
model = MyModel(n_embed, vocab_size)

In [127]:
model.compile(optimizer='adam',
                loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                metrics=['accuracy', tf.keras.metrics.F1Score()])

In [128]:
model.fit([x_pos_train, x_train], y_train, epochs=20, batch_size=512, validation_data=([x_pos_val, x_val], y_val))

Epoch 1/20
507/507 [==============================] - 74s 128ms/step - loss: 0.2443 - accuracy: 0.9089 - f1_score: 0.5550 - val_loss: 0.2458 - val_accuracy: 0.9037 - val_f1_score: 0.7127
Epoch 2/20
507/507 [==============================] - 73s 145ms/step - loss: 0.2173 - accuracy: 0.9215 - f1_score: 0.6784 - val_loss: 0.2195 - val_accuracy: 0.9188 - val_f1_score: 0.6990
Epoch 3/20
507/507 [==============================] - 102s 201ms/step - loss: 0.2083 - accuracy: 0.9240 - f1_score: 0.6897 - val_loss: 0.2239 - val_accuracy: 0.9143 - val_f1_score: 0.7075
Epoch 4/20
507/507 [==============================] - 131s 259ms/step - loss: 0.2061 - accuracy: 0.9249 - f1_score: 0.6939 - val_loss: 0.2175 - val_accuracy: 0.9195 - val_f1_score: 0.6995
Epoch 5/20
507/507 [==============================] - 145s 286ms/step - loss: 0.2043 - accuracy: 0.9254 - f1_score: 0.6952 - val_loss: 0.2184 - val_accuracy: 0.9194 - val_f1_score: 0.7002
Epoch 6/20
507/507 [==============================] - 148s 293

In [113]:
x_train = df_final_train.drop(["label", "energy"], axis=1).to_numpy()
y_train = df_final_train["label"].to_numpy()
x_val = df_final_val.drop(["label", "energy"], axis=1).to_numpy()
y_val = df_final_val["label"].to_numpy()

In [114]:
model = xgb.XGBClassifier(n_estimators=1000, max_depth=10, learning_rate=0.01, n_jobs=-1)

In [115]:
model.fit(x_train, y_train, eval_set=[(x_val, y_val)], eval_metric='logloss', early_stopping_rounds=10)

[0]	validation_0-logloss:0.68593
[1]	validation_0-logloss:0.67889
[2]	validation_0-logloss:0.67196
[3]	validation_0-logloss:0.66519
[4]	validation_0-logloss:0.65851
[5]	validation_0-logloss:0.65201
[6]	validation_0-logloss:0.64559
[7]	validation_0-logloss:0.63929
[8]	validation_0-logloss:0.63313
[9]	validation_0-logloss:0.62705
[10]	validation_0-logloss:0.62111
[11]	validation_0-logloss:0.61525
[12]	validation_0-logloss:0.60953
[13]	validation_0-logloss:0.60388
[14]	validation_0-logloss:0.59833
[15]	validation_0-logloss:0.59290
[16]	validation_0-logloss:0.58755
[17]	validation_0-logloss:0.58229
[18]	validation_0-logloss:0.57712
[19]	validation_0-logloss:0.57205
[20]	validation_0-logloss:0.56707
[21]	validation_0-logloss:0.56214
[22]	validation_0-logloss:0.55731
[23]	validation_0-logloss:0.55256
[24]	validation_0-logloss:0.54791
[25]	validation_0-logloss:0.54332
[26]	validation_0-logloss:0.53879
[27]	validation_0-logloss:0.53435
[28]	validation_0-logloss:0.52999
[29]	validation_0-loglos

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.01, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=10, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              n_estimators=1000, n_jobs=-1, num_parallel_tree=None,
              predictor=None, random_state=None, ...)

In [116]:
y_pred = model.predict(x_val)
accuracy_score(y_val, y_pred), f1_score(y_val, y_pred)

(0.9216979114690997, 0.45184630738522946)